In [0]:
%run ../../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import col, count, when


# Camada GOLD

Na camada gold, as limpezas e ajustes já foram feitos, então essa camada é responsável por aplicar regras de negócio, agregações e junções de dados que convirjam para analises.

Essa é uma tabela sumarizada analítica.

O objetivo dela é informar os meses em que tiveram mais pedidos

In [0]:
tb_name = "gold.olist_multiple_orders"
dataset_location = "olist_multiple_orders_dataset"
target_location = f"dbfs:/FileStore/delta/gold/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion


In [0]:
df = spark.read.table("gold.olist_customer_orders") # leituira da delta table central, orders (ja da propria gold)


## 2 - preparation

In [0]:
df = df.groupBy("customer_unique_id").agg(
    count("order_id").alias("total_orders"),
)


# 2.1 multiple_orders?

informar se o cliente comprou mais de uma vez

In [0]:
df = df.withColumn("multiple_order", when(col("total_orders") > 1, True).otherwise(False))

In [0]:
display(df.take(10))

customer_unique_id,total_orders,multiple_order
969cdc8af5b07074766e79c1e6d76c24,1,false
f2a9bc9a1db05c873e1419654f747f9e,1,false
d0c5e56e04e886e73c79b3b71f0d6f0b,1,false
5f03b965e26e79a371d229cfeeb578d7,1,false
14a188558af6cd5bc222ca773d395a7f,2,true
ef1c2fafea5285a4bfb61386bc5e3154,1,false
d339ed835c9d8fd6b1e3c5cb60850bc5,1,false
39d6e50625a51a618c2cf02a026231c1,1,false
8d2fa65d968da66afc05f2e28250c9dc,1,false
d04921557f1cde4963c30a11dfb1719d,1,false



## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)

[LOG] Saving gold.olist_multiple_orders delta on dbfs:/FileStore/delta/gold/brazilian_ecommerce/olist_multiple_orders_dataset... OK!



## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

[LOG] Creating delta table gold.olist_multiple_orders on dbfs:/FileStore/delta/gold/brazilian_ecommerce/olist_multiple_orders_dataset... OK!


In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from gold.olist_multiple_orders limit 10

customer_unique_id,total_orders,multiple_order
969cdc8af5b07074766e79c1e6d76c24,1,false
f2a9bc9a1db05c873e1419654f747f9e,1,false
d0c5e56e04e886e73c79b3b71f0d6f0b,1,false
5f03b965e26e79a371d229cfeeb578d7,1,false
14a188558af6cd5bc222ca773d395a7f,2,true
ef1c2fafea5285a4bfb61386bc5e3154,1,false
d339ed835c9d8fd6b1e3c5cb60850bc5,1,false
39d6e50625a51a618c2cf02a026231c1,1,false
8d2fa65d968da66afc05f2e28250c9dc,1,false
d04921557f1cde4963c30a11dfb1719d,1,false
